In [14]:
import pandas as pd
from io import StringIO

output = StringIO()
pd.read_csv('./data/raw_data/area/v_region_2023.csv')[['REG', 'LIBELLE', 'CHEFLIEU']].rename(
    columns={'REG': 'code', 'LIBELLE': 'nom', 'CHEFLIEU': 'cheflieu'}).to_csv(output, index=False)

output.seek(0)
output.read()

"code,nom,cheflieu\r\n1,Guadeloupe,97105\r\n2,Martinique,97209\r\n3,Guyane,97302\r\n4,La Réunion,97411\r\n6,Mayotte,97608\r\n11,Île-de-France,75056\r\n24,Centre-Val de Loire,45234\r\n27,Bourgogne-Franche-Comté,21231\r\n28,Normandie,76540\r\n32,Hauts-de-France,59350\r\n44,Grand Est,67482\r\n52,Pays de la Loire,44109\r\n53,Bretagne,35238\r\n75,Nouvelle-Aquitaine,33063\r\n76,Occitanie,31555\r\n84,Auvergne-Rhône-Alpes,69123\r\n93,Provence-Alpes-Côte d'Azur,13055\r\n94,Corse,2A004\r\n"

In [19]:
import tomllib

with open('./data/config.toml', 'rb') as f:
    data = tomllib.load(f)
    print(data)

{'database': {'dbname': 'BDA', 'user': 'vscode', 'password': 'SQL_BDA', 'host': 'localhost', 'port': 5432}, 'region': {'REG': 'code', 'LIBELLE': 'nom', 'CHEFLIEU': 'cheflieu'}}


In [5]:
'coucou'.find('ou', 5)

-1

In [12]:
import re

string = "Hello world; welcome to the world of Python;"
substring = "world"

a = [a.start() for a in re.finditer(substring, string)]
for match in a:
    print(match.start())

b = [a.start() for a in re.finditer(';', string)]
for match in b:
    print(match.start())

print(string[a[0]:b[0]])

6
28
12
44


TypeError: 'callable_iterator' object is not subscriptable